In [1]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np

In [2]:
msms = pd.read_csv("/cmnfs/data/proteomics/MQ_cit_LeeLab/MQ100%/msms.txt", sep = "\t")

/tmp/ipykernel_53149/3099777723.py:1: DtypeWarning: Columns (69,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  msms = pd.read_csv("/cmnfs/data/proteomics/MQ_cit_LeeLab/MQ100%/msms.txt", sep = "\t")


In [3]:
def check_citrullination(modifications):
    if isinstance(modifications, list):
        return 'Citrullination' in modifications
    return 'Citrullination' == modifications

In [4]:
# Applying the conditions
condition = msms['Reverse'].isna() & msms['Modifications'].apply(check_citrullination)

# Checking if any row satisfies both conditions
if condition.any():
    print("There is at least one entry with '+' in 'Reverse' and 'Citrullination' in 'Modifications'.")
else:
    print("No entry found with both conditions.")

There is at least one entry with '+' in 'Reverse' and 'Citrullination' in 'Modifications'.


In [5]:
# Creating a new DataFrame with the filtered rows
filtered_df = msms[condition]

In [6]:
condition2 = msms['Reverse'].str.contains(r'\+') & msms['Modifications'].apply(check_citrullination)
filtered_df2 = msms[condition2]

In [7]:
msms[msms['Modifications'].apply(check_citrullination)].shape

(4542, 73)

In [8]:
msms.shape

(408354, 73)

In [9]:
new_msms = pd.read_csv("/cmnfs/proj/bmpc_dlomix/oktoberfest_output/citrullination_test2/msms/msms.prosit")

In [10]:
new_msms.shape

(385052, 10)

In [11]:
any(["UNIMOD:7" in peptide for peptide in new_msms["MODIFIED_SEQUENCE"]])

False

In [12]:
any(["R(Citrullination)" in peptide for peptide in new_msms["MODIFIED_SEQUENCE"]])

False

In original msms, replace R(Citrullination) with R(ci) to avoid conflict with C -> C[UNIMOD:4]

In [15]:
msms['Modified sequence'] = msms['Modified sequence'].str.replace(r'R\(Citrullination\)', 'R(ci)', regex=True)


In [16]:
msms["Modified sequence"].unique()

array(['_(Acetyl (Protein N-term))AAAAAAAAAAGAAGGR_',
       '_AAAAAAAAAVSR(ci)R(ci)R(ci)_', '_AAAAAAALQAK_', ..., '_YYYSGKR_',
       '_YYYYHRQYRR_', '_YYYYWHLR_'], dtype=object)

In [19]:
msms.to_csv("/cmnfs/proj/bmpc_dlomix/datasets/citrullination_100_replaced/msms.txt", sep = "\t")